In [1]:
%matplotlib inline
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import set_option
import string
import re
set_option('display.max_rows', 16)
LARGE_FIGSIZE = (12, 8)

In [4]:
apath = 'C:\\Jeyaraj\\Analysis\\SFDC_Dump\\FY16Q4M3_Funnel\\'

In [36]:
file_name = apath+'Compiled_FY16Q4M3_Funnel.xlsx'
sheet_name = 'Filtered2'
xl = pd.ExcelFile(file_name)
df_chan_arch = xl.parse(sheet_name)

In [41]:
file_name = apath+'funnel_report_2016-07-12.xlsx'
sheet_name = 'Com_FY16Q4M3'
xl = pd.ExcelFile(file_name)
df_sfdc = xl.parse(sheet_name)

In [48]:
change_cols = {
    'RM Name': 'Opportunity Owner',
    'Vertical': 'Level 05 Territory',
    'Segment': 'Level 04 Territory',
    'Partner': 'Primary Partner',
    'Week': 'Fiscal Week of Month',
    'Arch': 'Technologies',
    'Discount': 'Expected Product Discount%',
    'Deal_ID': 'Deal ID'
}
df_chan_arch.rename(columns=change_cols, inplace=True)
df_chan_arch.columns

Index([                         u'Sr_No',                     u'From_Where',
                        u'Compare_Level',              u'Opportunity Owner',
                             u'PAM Name',             u'Level 04 Territory',
                   u'Level 05 Territory',                u'Primary Partner',
                         u'Account Name',            u'Unique Account Name',
                      u'Forecast Status', u'Expected Product $ (converted)',
       u'Expected Service $ (converted)',            u'Products / Services',
                                u'Month',           u'Fiscal Week of Month',
                         u'Technologies',                        u'Deal ID',
           u'Expected Product Discount%',                        u'Remarks',
                            u'File_Name'],
      dtype='object')

In [49]:
df_chan_arch['Level 01 Territory'] = 'APJ'
df_chan_arch['Level 02 Territory'] = 'INDIA_AREA'
df_chan_arch['Level 03 Territory'] = 'INDIA_COMM_1'
df_chan_arch['Fiscal Period'] = 'Q4-2016'
df_chan_arch['Fiscal Quarter'] = '4'
df_chan_arch['Fiscal Year'] = 2016
map_week = {
    'W3': 3,
    'W4': 4,
    'W5': 5
}
map_l4 = {
    'EU1': 'INDIA_COMM_EU1_SOUTH',
    'EU2': 'INDIA_COMM_EU2_WEST',
    'EU3': 'INDIA_COMM_EU3_NE_SAARC',
    'CBN': 'CBN',
    'Ent': 'ENT',
    'POS': 'POS'
}
df_chan_arch['Level 04 Territory'] = df_chan_arch['Level 04 Territory'].map(map_l4)
df_chan_arch['Fiscal Week of Month'] = df_chan_arch['Fiscal Week of Month'].map(map_week)

In [51]:
df_chan_arch['Level 02 Territory'] = 'INDIA_AREA'
df_chan_arch.columns

Index([                         u'Sr_No',                     u'From_Where',
                        u'Compare_Level',              u'Opportunity Owner',
                             u'PAM Name',             u'Level 04 Territory',
                   u'Level 05 Territory',                u'Primary Partner',
                         u'Account Name',            u'Unique Account Name',
                      u'Forecast Status', u'Expected Product $ (converted)',
       u'Expected Service $ (converted)',            u'Products / Services',
                                u'Month',           u'Fiscal Week of Month',
                         u'Technologies',                        u'Deal ID',
           u'Expected Product Discount%',                        u'Remarks',
                            u'File_Name',             u'Level 01 Territory',
                   u'Level 02 Territory',             u'Level 03 Territory',
                        u'Fiscal Period',                 u'Fiscal Quarter',

In [45]:
df_sfdc['File_Name'] = 'funnel_report_2016-07-12'
df_sfdc['From_Where'] = 'SFDC'
df_sfdc['Compare_Level'] = 'SFDC_FUNNEL'
df_sfdc.columns
df_sfdc.head()

Index([                                  u'Sr_No',
                           u'Sales Coverage Code',
                            u'Level 01 Territory',
                            u'Level 02 Territory',
                            u'Level 03 Territory',
                            u'Level 04 Territory',
                            u'Level 05 Territory',
                            u'Level 06 Territory',
                            u'Level 07 Territory',
                             u'Opportunity Owner',
                       u'Opportunity Owner Alias',
                          u'Forecasting Position',
                         u'Forecasting Positions',
                                  u'Account Name',
                           u'Unique Account Name',
                        u'Account Name (English)',
                               u'Primary Partner',
                                  u'Has Partners',
                                     u'Partner 2',
                               

In [52]:
df = pd.concat([df_sfdc,df_chan_arch])

In [53]:
writer = pd.ExcelWriter(apath+'Funnel_Findings.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='SFDC+CHAN&ARCHS')
df_chan_arch.to_excel(writer, sheet_name='CHAN&ARCHS')
df_sfdc.to_excel(writer, sheet_name='SFDC')
writer.save()

In [27]:
#df_funnel.groupby(['Unique End Customer', 'From_Where'])['Commit'].sum()
df_commit = df_funnel[pd.notnull(df_funnel.Commit)]
df_commit_no_duplicates = df_commit.drop_duplicates(['Unique End Customer', 'Commit'], keep=False)

In [30]:
df_commit_duplicates = pd.concat([df_commit_no_duplicates, df_commit]).drop_duplicates('Sr_No', keep=False)

In [33]:
df_upside = df_funnel[pd.notnull(df_funnel.Upside)]
df_upside_no_duplicates = df_upside.drop_duplicates(['Unique End Customer', 'Upside'], keep=False)

In [34]:
df_upside_duplicates = pd.concat([df_upside_no_duplicates, df_upside]).drop_duplicates('Sr_No', keep=False)

In [35]:
writer = pd.ExcelWriter(apath+'Funnel_Findings.xlsx', engine='xlsxwriter')
df_commit_no_duplicates.to_excel(writer, sheet_name='Commit_No_Dup')
df_commit_duplicates.to_excel(writer, sheet_name='Commit_Dup')
df_upside_no_duplicates.to_excel(writer, sheet_name='Upside_No_Dup')
df_upside_duplicates.to_excel(writer, sheet_name='Upside_Dup')
writer.save()